# Week 3: Representational Similarity Analysis (RSA)
This week's tutorial is about RSA! 

## About this week's lab
We'll be looking at how to transform patterns into RDMs using various distance measures, how to test the relation between "feature RDMs" and "brain RDMs", and take a look at exploratory RDM visualization using multidimensional-scaling (MDS).

## Contents
1. Loading in the data
2. Preprocessing
3. Neural RDMs
4. Exploratory analysis using MDS
5. Categorical RDms
6. Continuous/computational RDMs
7. Testing RDMs

**Estimated time needed to complete**: 8-12 hours

In [ ]:
# Some imports for the rest of the tutorial
import os
import numpy as np
import nibabel as nib
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
from nilearn import image, datasets, plotting, masking
import pandas as pd
%matplotlib inline

## 1. Loading in the data
In this notebook, we are going to work with real data straightaway! For now, we'll work only with the data from a single run, though. In addition, to not use too much RAM, we'll analyze only the data from an ROI. In contrast to last week, we are going to use a "functional ROI" based on the localizer data from our "flocBLOCKED" task. To derive a functional ROI, we already computed (for each subject) multiple contrasts and the associated whole-brain $z$-score maps in both subject "native" space (*T1w*) and standard space (*MNI152NLin2009cAsym*):

In [ ]:
from niedu.global_utils import get_data_dir
data_dir = get_data_dir()
floc_dir = os.path.join(data_dir, 'derivatives', 'floc', 'sub-03', 'rois')
print("We have the following maps:\n-", '\n- '.join(os.listdir(floc_dir)))

These maps have been computed using a "condition > other conditions". We are, of course, going to use the "face > (place, character, body, object)" map for our ROI and use the data in MNI space:

In [ ]:
floc_roi = os.path.join(floc_dir, 'sub-03_task-flocBLOCKED_space-MNI152NLin2009cAsym_desc-face_zscore.nii.gz')

# Let's plot the unthresholded map as well
plotting.plot_stat_map(floc_roi, cut_coords=(40, -46, -20))

As you can see in the plot above, this subject shows a strong response to faces (relative to other conditions) in right temporal lobe, just where you'd expect to find the fusiform face area (FFA). But to derive an ROI from this map, we should somehow binarize this image. While this choice is somewhat arbitrary, let's threshold this map at $z > 3$:

In [ ]:
from nistats.thresholding import map_threshold
floc_roi_bin = image.math_img('(img > 3).astype(int)', img = floc_roi)
plotting.plot_roi(floc_roi_bin, cut_coords=(40, -46, -20));

The current ROI, however, contains *a lot* of voxels, also many far outside the location of where we'd expect the FFA. One "trick" we can use to further restrict the number of voxels is to constrain our functional ROI to a particular anatomical location. Here, we'll pick the right temporal ocipital fusiform (rTOF):

In [ ]:
ho_atlas = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm', symmetric_split=True)
ho_map = ho_atlas['maps']
rTOF_idx = ho_atlas['labels'].index('Right Temporal Occipital Fusiform Cortex')
rTOF_roi = nib.Nifti1Image((ho_map.get_fdata() == rTOF_idx).astype(int), ho_map.affine)
plotting.plot_roi(rTOF_roi, cut_coords=(40, -46, -20));

Note that the anatomical ROI has a slightly higher spatial resolution (2 mm$^2$, instead of $2.7 \times 2.7 \times 2.97$). As such, we need to resample the anatomical mask to our functional resolution:

In [ ]:
rTOF_roi_resamp = image.resample_to_img(rTOF_roi, floc_roi_bin, interpolation='nearest')

Finally, we can intersect the two masks using the `intersect_masks` from the Nilearn `masking` module:

In [ ]:
# Setting the threshold to 1 means: only select voxels that are in *both* masks
ffa_mask = masking.intersect_masks((floc_roi_bin, rTOF_roi_resamp), threshold=1)
plotting.plot_roi(ffa_mask, cut_coords=(40, -46, -20));

Alright, now let's load in the single trial (univariate noise normalized) patterns from run 1:

In [ ]:
nibs_dir = os.path.join(data_dir, 'derivatives', 'nibetaseries_lsa', 'sub-03', 'ses-1', 'func')
files = sorted(glob(os.path.join(nibs_dir, '*task-face*run-1*space-MNI*STIM*')))

# Concatenate images into 4D array and immediately mask it
R = masking.apply_mask(image.concat_imgs(files), ffa_mask)
print("Shape of R:", R.shape)

plt.imshow(R, aspect='auto')
plt.xlabel("Voxels")
plt.ylabel("Samples (trials)")
plt.title(r"$\mathbf{R}$", fontsize=20)
plt.colorbar()
plt.show()

Lastly, we need some experimental variable(s) to relate to this brain pattern. Let's load in our events file:

In [ ]:
import pandas as pd
events_file = os.path.join(data_dir, 'sub-03', 'ses-1', 'func', 'sub-03_ses-1_task-face_acq-Mb4Mm27Tr700_run-1_events.tsv')
events_df = pd.read_csv(events_file, sep='\t')

# Let's remove the rating/response events
# The .query method is great for filtering!
events_df = events_df.query("trial_type != 'rating' and trial_type != 'response'")
events_df

There are many different experimental features that we'd could use for our analysis, but for now, we'll stick with a single categorical (binary) one: face sex ("male" or "female"). 

In [ ]:
S = events_df['face_sex'].to_numpy()
print(S)

<div class='alert alert-warning'>
    <b>ToDo</b> (1 point): For our analysis, we'll need the labels ($S$) in numeric format. Convert the string labels ("male", "female") to a numeric format (male: 1, female: 0) and store the result in a variable called <tt>S_num</tt> (which should be a numpy array).
</div>

In [ ]:
''' Implement your ToDo here. '''

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. '''
from niedu.tests.nipa.week_3 import test_str2num
test_str2num(S, S_num)

## 2. Preprocessing
In terms of preprocessing, there are a couple of things that you need to keep in mind when planning to use representational similarity analyses. First, while standardization (ensuring zero mean and unit standard deviation for each brain feature) is a common preprocessing step in decoding analyses, it is somewhat of a controversial for RSA (see e.g. [this article](https://www.frontiersin.org/articles/10.3389/fnins.2013.00174/full)). As such, we are not going to apply standardization to our data.

### 2.1. Multivariate noise normalization
In week 1, we discussed univariate noise normalization, i.e., dividing each brain feature's activity estimate ($\hat{\beta}$) by the standard deviation of the noise ($\hat{\sigma}$), which allows you to "downweigh" noisy voxels. Specifically for RSA, some people use *multivariate* noise normalization, which additionally incorporates the noise *covariance* between voxels. Importantly, both normalization methods use the $K \times K$ variance-covariance matrix derived from the $T \times K$ residuals. 

So, before going on, let's first load in the residuals from run 1 (which is also conveniently returned by `nibetaseries`):

In [ ]:
resids_path = os.path.join(nibs_dir, 'sub-03_ses-1_task-face_acq-Mb4Mm27Tr700_run-1_space-MNI152NLin2009cAsym_desc-residuals_bold.nii.gz')
# We immediately apply the FFA mask to the residuals
resids = masking.apply_mask(resids_path, ffa_mask)
print("Shape of masked residuals (TxK):", resids.shape)

In week 1, we computed the voxelwise noise standard deviation by simply called the numpy `std` function (or method) on our time axis:

In [ ]:
noise_std = np.std(resids, axis=0)
plt.plot(noise_std)
plt.xlabel("Voxel", fontsize=15)
plt.ylabel(r'$\hat{\sigma}$', fontsize=20)
sns.despine()
plt.show()

However, we can also get the voxelwise noise standard deviation by computing the noise variance-covariance matrix and extracting the (square root of the) diagonal, because the diagonal represents the variance of the voxels (the "covariance with itself", so to say).

Let's first compute the covariance matrix:

In [ ]:
# We need to transpose (.T) the residuals,
# otherwise we'd get a TxT covariance matrix
noise_cov = np.cov(resids.T, bias=True)
plt.imshow(noise_cov, vmin=-1, vmax=20)
plt.title("Noise covariance matrix", fontsize=18)
plt.xlabel('Voxels', fontsize=15)
plt.ylabel('Voxels', fontsize=15)
plt.colorbar()
plt.show()

Just to convince you that the (square root of the) diagonal is the same as the standard deviation we computed earlier:

In [ ]:
noise_std_from_cov = np.sqrt(np.diag(noise_cov))
fig, axes = plt.subplots(ncols=2, figsize=(15, 4))
axes[0].plot(noise_std)
axes[1].plot(noise_std_from_cov, c='tab:orange')
axes[0].set_xlabel("Voxel", fontsize=15)
axes[1].set_xlabel("Voxel", fontsize=15)
axes[0].set_ylabel(r'$\hat{\sigma}$', fontsize=20)
fig.tight_layout()
sns.despine()
fig.show()

The reason we need the variance-covariance matrix is that for *multivariate* noise normalization, we use the *full* matrix, i.e., including the off-diagonal elements (the covariance between voxels). To do so, we first need to compute the *whitening* matrix, which is often denoted by $D$ and is computed by taking the square root of the inverse of the estimated variance-covariance matrix ($\hat{\Sigma}$):

\begin{align}
D = \hat{\Sigma}^{-\frac{1}{2}}
\end{align}

Whitening using this particular whitening matrix ($\hat{\Sigma}^{-\frac{1}{2}}$) is also called [ZCA or Mahalanobis whitening](https://en.wikipedia.org/wiki/Whitening_transformation). To apply this whitening matrix to our patterns ($\mathbf{R}$), we simply take the dot product between the patterns and the whitening matrix:

\begin{align}
R_{\mathrm{mnn}} = RD
\end{align}

where $R_{\mathrm{mnn}}$ is the multivariate noise normalized pattern matrix. Let's first compute the whitening matrix. We can use the matrix square root function `sqrtm` from the `scipy.linalg` package:

In [ ]:
# square root of inv = ^(-1/2)
from scipy.linalg import sqrtm
D = sqrtm(np.linalg.inv(noise_cov))

And to get the multivariate noise normalized patterns, we compute the dot product:

In [ ]:
R_mnn = R @ D

Let's visualize the unnormalized, univariate noise normalized, and multivariate noise normalized patterns:

In [ ]:
R_unn = R / noise_std

fig, axes = plt.subplots(ncols=3, figsize=(13, 3))
axes[0].imshow(R)
axes[0].set_title("No normalization", fontsize=15)
axes[1].imshow(R_unn)
axes[1].set_title("UNN", fontsize=15)
axes[2].imshow(R_mnn)
axes[2].set_title("NNN", fontsize=15)
for i in range(3):
    axes[i].set_xlabel("Voxels", fontsize=15)
axes[0].set_ylabel("Trials", fontsize=15)
fig.tight_layout()
fig.show()

Here, our patterns consist of only 64 voxels. Often, however, you might want to use patterns with (many) more voxels. For multivariate noise normalization, which uses the full $K \times K$ variance-covariance matrix, using more brain features ($K$) than samples ($N$) often lead to a very unstable variance-covariance matrix (or in technical terms, a matrix that is not "positive semidefinite".). One trick that makes the variance-covariance matrix estimation more stable is to apply <em>regularization</em> (sometimes called "shrinkage"). This regularization will "shrink" the matrix more towards the identity matrix ($I$, i.e., a matrix with all zeros except the diagonal, which contains ones) when the ratio between brain features and samples becomes larger.<br>
    
One such shrinkage method is the "Ledoit-Wolf" covariance estimator. Below, we import this function:

In [ ]:
from sklearn.covariance import ledoit_wolf

<div class='alert alert-warning'>
    <b>ToDo</b> (1 point): Read through the <a href="https://scikit-learn.org/stable/modules/generated/sklearn.covariance.ledoit_wolf.html#sklearn.covariance.ledoit_wolf">documentation of the ledoit_wolf function</a> and then use it to compute the regularized covariance matrix of our pattern matrix and subsequently use this to multivariate noise normalize our pattern matrix. Store the result in a new variable called <tt>R_mnn_reg</tt>.
</div>

In [ ]:
''' Implement your ToDo here. '''
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the ToDo above. '''
from niedu.tests.nipa.week_3 import test_ledoit_wolf    
test_ledoit_wolf(R, resids, R_mnn_reg)

### 2.2. Pattern drift
"Pattern drift" is a phenomenon in which your patterns (i.e., rows of $\mathbf{R}$) are correlated in time: the pattern of trial 1 (strongly) correlates with trial 2, the pattern of trial 2 (strongly) correlates with trial 3, etc. This pattern drift (or "autocorrelation") seems to occur regardless of whether you remove the low-frequency components during preprocessing or first-level modelling (e.g., using a discrete cosine basis set or other high-pass filter).

In fact, the single patterns in the NI-edu dataset also show quite some pattern drift (as can be seen in the previous figure)! 

There are several ways to deal with pattern drift (which are described in detail in [this paper](https://www.biorxiv.org/content/10.1101/032391v2)). The most promising one involves computing distances between trials (or conditions) from *different* runs; for example, to get the distance between trial A and B, compute $\mathrm{distance}[R^{A}_\mathrm{run\ 1}, R^{B}_\mathrm{run\ 2}]$. As these between-run distances are a bit complicated to compute (in terms of code), we won't discuss their implementation here.

## 3. Neural RDMs
The first step in a representational similarity analysis is to create a "neural representational dissimilarity matrix" (RDM). This matrix is a symmetric $N \times N$ matrix which represents how "dissimilar" patterns of different samples (i.e., rows in your pattern matrix, $\mathbf{R}$). Note that the samples could be trials (e.g., "face 1", "face 2", "face 3", etc.) but could also be conditions (e.g., "faces", "houses", "objects", etc.). In our case, we're focusing on trials.

For example, suppose we have only data from four trials (i.e., four rows in our pattern matrix $\mathbf{R}$). A corresponding $4\times 4$ RDM could like the following:

In [ ]:
# some made up RDM
example_rdm = np.array([
    [0, 7.2, 1.2, 5.2],
    [7.2, 0, 3.1, 4.7],
    [1.2, 3.1, 0, 6.5],
    [5.2, 4.7, 6.5, 0]
])

plt.imshow(example_rdm)
plt.yticks(np.arange(4))
plt.xlabel("Trials", fontsize=15)
plt.ylabel("Trials", fontsize=15)
plt.title("Example RDM", fontsize=20)
cbar = plt.colorbar()
cbar.ax.set_ylabel('Dissimilarity', fontsize=15)
plt.show()

In this example RDM, each cell represents a particular distance between two patterns. For example, the cell in the bottom left corner represents the dissimilarity between trial 1 and trial 4. Note that the RDM is symmetric, because the dissimilarity between trial 1 and 4 is the same as trial 4 and 1; also, the cells on the diagonal are all zero, as they represent the distance between a particular pattern and itself, which is zero!  Note that these two properties (symmetricity, zero diagonal) are only true when you compute your RDM on trials within the same run (unlike the between-run pattern distances, discussed previously).

For now, we'll stick with within-run RDMs (as they're a little easier to compute).

In a way, you can think about RDMs as "inverse" correlation matrices in which cells do not represent correlations (a kind of *similarity* metric) but distances.

By now, you might ask youself: "but how do you actually compute these dissimilarities?" Well, this depends on what *distance metric* you use! There are many different functions you can use to quantify the dissimilarity between two vectors (i.e., two rows in our pattern matrix). Actually, reflecting the intuition that an RDM is basically the inverse of a correlation matrix, one metric that is sometimes used is the $1-r$ distance\*. This distance simply quantifies distance as the 1 minus the correlation between two patterns. For example, the $1-r$ distance between "pattern A" and "pattern B" is $1-\mathrm{corr(pattern\ A, pattern\ B})$.

---
\* In the RSA literature, some people use the *cosine distance*, which is the angle between two vectors; when the patterns are mean centered (i.e., the rows in $\mathbf{R}$ have a mean of 0), this is exactly the same as the $1-r$ distance!

<div class='alert alert-warning'>
    <b>ToDo</b> (1 point): For our data (the variable <tt>R</tt>), compute the RDM using the $1 -r$ distance and store it in a variable named <tt>rdm_1minr</tt>. Note: no need for a for-loop! (You might want to check out the <tt>np.corrcoef</tt> function.)
</div>

In [ ]:
''' Implement your ToDo here. '''

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. '''
from niedu.tests.nipa.week_3 import test_1minr    
test_1minr(R, rdm_1minr)

Another often-used distance metric used for RDMs, and perhaps the most intuitive one, is the Euclidean distance. This distance is computed as the square root of the sum of squared distances between two patterns (e.g., $p$ and $q$) consisting of $K$ elements:

\begin{align}
\delta_{euclidean} = \sqrt{\sum_{j=1}^{K}{(p_{j} - q_{j})^{2}}}
\end{align}

Below, we define two example patterns with four features and compute the Euclidean distance between them:

In [ ]:
p = np.array([1, 3, 8, -4])
q = np.array([5, -3, 2, 1])

# No need for a for loop!
euc_dist = np.sqrt(np.sum((p - q)**2))
print("Euclidean distance between p and q:", euc_dist)

<div class='alert alert-warning'>
    <b>ToDo</b> (2 points): While all pairwise distances were easy to calculate using the $1-r$ distance, it takes a little more code to do this using the Euclidean distance. Compute the RDM based on the Euclidean distance for our data (the variable <tt>R</tt>) and store this in a variable named <tt>rdm_euc</tt>. Also visualize the RDM using the pyplot <tt>imshow</tt> function. Unless you're a linear algebra wizard, you need to use for loops to compute the RDM. Do not use any external functions (beyond numpy). Hint: pre-allocate your RDM first and then fill it cell by cell in a nested for loop.
</div>

In [ ]:
''' Implement your ToDo here. '''

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. '''
from sklearn.metrics import euclidean_distances
np.testing.assert_array_almost_equal(euclidean_distances(R), rdm_euc, decimal=3)
print("Well done!")

As you might have seen in the test cell, scikit-learn actually provides several functions to quickly compute distance matrices (RDMs) using various distance metrics. We recommend using the generic `pairwise_distances` function:

In [ ]:
from sklearn.metrics import pairwise_distances

To compute an $N\times N$ distance matrix from a $N\times K$ pattern array ($\mathbf{R}$), you can use it as follows:
```python
rdm = pairwise_distances(R, metric='name_of_metric')
```

For example, to compute an RDM based on the "cosine" distance (which is similar to the $1-r$ distance), you can run:

In [ ]:
rdm_cosine = pairwise_distances(R, metric='cosine')

plt.imshow(rdm_cosine)
plt.xlabel("Trials", fontsize=15)
plt.ylabel("Trials", fontsize=15)
plt.title("Cosine-based RDM", fontsize=20)
cbar = plt.colorbar()
cbar.ax.set_ylabel('Cosine distance', fontsize=15)
plt.show()

Note that there is no agreed-upon "best" distance metric! If you want to know more about the different (dis)similarity metrics for pattern analyses, check out [this article](https://link.springer.com/article/10.1007/s42113-019-00068-5).

For the next couple of sections, we'll use the Euclidean distance-based RDM:

In [ ]:
rdm_R = pairwise_distances(R, metric='euclidean')

<div class='alert alert-info'>
    <b>ToThink</b> (1 point): Euclidean and $1-r$/cosine distances differ in one major aspect related to the "type" of information they can encode/pick up. What do you think this is?
</div>

YOUR ANSWER HERE

## 4. Exploratory analysis using MDS
Most applications of RSA involve relating *neural* RDMs with RDMs based on experimental features (which we'll discuss in the next section). However, you can also do exploratory analyses on your neural RDM only! This is usually done by investigating the (dis)similarity structure (or "representational geometry" in RSA terms), usually in a 2D or 3D space. 

As pattern analyses are often applied to very high-dimensional data (i.e., patterns with many brain features, $K$), people often project the data (i.e., the patterns) into a lower dimensional space. We already encountered one such method in week 1: PCA! However, when interested in the (dis)similarity structure of your data, *multidimensional scaling* (MDS) is more appropriate. Just like PCA, this technique aims to create combinations of features into a lower-dimensional subset of components, such that the high-dimensional distances are presented as much as possible in the lower-dimensional space. For example, if the distance between A and B is 436 in high-dimensional space (e.g., $K=500$), MDS tries to create a lower-dimensional space (usually 2 components) in which the distance between A and B is as close as possible to 436 (as well as all other distances between patterns).

Of course, scikit-learn contains an implementation of MDS that uses the familiar `fit`/`transform` methods. Importantly, it can take in a $N\times K$ matrix (like our pattern matrix $\mathbf{R}$) and compute the high-dimensional distance structure (i.e., the RDM) internally or you can give it your precomputed RDM. In the latter case, you need to initialize it with `dissimilarity='prepcomputed'`, which is what we're going to do:

In [ ]:
from sklearn.manifold import MDS
mds = MDS(dissimilarity='precomputed', n_components=2)

Then, we call the `fit_transform` method to compute lower-dimensional ($K=2$) representation of the data (which is, again, an $N \times K$ array, but this time, $K=2$!):

In [ ]:
mds_R = mds.fit_transform(rdm_R)
print("Shape of mds_R:", mds_R.shape)
plt.figure(figsize=(8, 5))
plt.grid()
plt.scatter(mds_R[:, 0], mds_R[:, 1])
plt.xlabel('MDS component 1', fontsize=20)
plt.ylabel('MDS component 1', fontsize=20)
plt.show()

<div class='alert alert-warning'>
    <b>ToDo</b> (1 point): Below, re-plot the MDS scatterplot, but this time, color the datapoints (i.e., the trial patterns) according to their trial onset (you can set the color of the points with <tt>c</tt> parameter in the <tt>scatter</tt> function), which you can extract from the <tt>events_df</tt> dataframe. This way, datapoints (i.e., patterns) with a similar onset have a similar hue.
</div>

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

<div class='alert alert-info'>
    <b>ToThink</b> (1 point): Which phenomenon (that we discussed before) is clearly visible in this low-dimensional embedding of the data? 
</div>

YOUR ANSWER HERE

Instead of coloring the datapoints according to some experimental property/feature (such as onset), an even more potent way to visualize MDS-embeddings is to plot the actual images that correspond with the trial patterns (here: the faces shown to the subject)! This way, you may find patterns in the data that you might not have thought of!

However, in our opinion, the true strength of RSA lies in their ability to test hypotheses about complex representational structures using experimental features, which is discussed next.

## 5. Categorical RDMs
In most pattern analyses, we'd like to evaluate the (possibly) association between experimental features and brain patterns. In RSA, this is done by comparing neural RDMs (discussed in the previous sections) with RDMs based on experimental features. These experimental feature RDMs (let's call them "feature RDMs") are constructed in largely the same way as neural RDMs: by computing the pairwise distance between samples!

The experimental features ($P$) that you use for your feature RDM of course depend on your hypothesis! Importantly, unlike decoding models, RSA naturally handles high-dimensional feature spaces very well. Like neural RDMs, no matter how many features you use, you'll always analyze the resulting $N\times N$ RDM! 

After constructing your feature RDM, you can test whether the "geometry" of your hypothesized feature space matches the geometry of your brain matters. In other words, you test whether the pattern of distances is similar in your brain data and your experimental features. Technically, you can use *any* (set of) feature(s) that you believe match the geometry of the corresponding brain patterns. How to actually test this will be discussed in section 5.

In this section, we'll focus on the most straightforward type of feature RDM: the categorical RDM. This RDM, basically, investigates whether patterns belonging to the same condition are more similar than patterns belonging to a different condition (note the similarity to decoding models). For example, we could hypothesize that the FFA represents face gender, which should accordingly lead to relatively small neural distances between images of the same face gender and relatively large neural distances between images of a different face gender. Before delving into how we should construct a corresponding feature RDM, let's first define our experimental feature: face gender. In the first section, we already extracted this from the events file. Now, let's convert it to a numeric format:

In [ ]:
from sklearn.preprocessing import LabelEncoder
face_gen = LabelEncoder().fit_transform(S)
print("Face gender, numeric:", face_gen)

Now, to capture this feature into an RDM, we can construct an RDM with zeros in cells corresponding to trials with the same condition (both male or both female faces) and ones everywhere else, which capture the hypothesis that trials should have a smaller distance when they are of the same condition (0) than when they are of a different condition (1). 

<div class='alert alert-warning'>
    <b>ToDo</b> (1 point): Create this face-gender RDM and store it in a variable named <tt>rdm_fg</tt>, an $N \times N$ numpy array. Do not use any external functions for this. 
</div>

In [ ]:
''' Implement your ToDo here. '''

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. '''
from niedu.tests.nipa.week_3 import test_rdm_fg
test_rdm_fg(face_gen, rdm_fg)

Technically, to create this categorical RDM (with only two levels), you can also use the `pairwise_distances` function with the "manhattan" metric (the sum of *absolute* distances):

In [ ]:
# Note the np.newaxis, which is needed because pairwise_distances
# assumes that the input is 2D
rdm_fg_pd = pairwise_distances(face_gen[:, np.newaxis], metric='manhattan')
plt.imshow(rdm_fg_pd)
plt.colorbar()
plt.title("Categorical RDM")
plt.show()

## 6. Continuous/computational RDMs
The categorical RDMs discussed in the previous section are the most simple implementation of feature RDMs, based on only a single categorical feature. RSA really shines, though, when relating more complex feature sets (sometimes called "feature spaces") consisting of multiple (continuous) variables. 

For example, for our data, we have subject-specific ratings of dominance, trustworthiness, and attractiveness for all of the faces shown to (the same) subjects. If we'd want to investigate whether a particular brain region represents these face properties (you might call them "[social judgements](https://www.sciencedirect.com/science/article/abs/pii/S0959438813000147)), we could create a feature RDM base on these three features; in fact, we'll do that below:

In [ ]:
sj = events_df.loc[:, ['subject_dominance', 'subject_trustworthiness', 'subject_attractiveness']].to_numpy()
rdm_sj = pairwise_distances(sj, metric='euclidean')

plt.imshow(rdm_sj)
plt.title("Social judgements RDM", fontsize=15)
plt.show()

Note though, that all information about the individual features (attractiveness, trustworthiness, and dominance) is lost in this RDM! The feature RDM should represent the geometry of the entire feature space, not the effects of the individual features on brain activity (although we discuss a technique that allows for this type of inference in the next section). 

<div class='alert alert-warning'>
    <b>ToDo</b> (2 point): Note that as long as you can formalize your hypothesis into a feature matrix ($N \times P$), you can create an RDM from it (and test it with RSA)! Suppose that I believe that social judgements are not represented in a bipolar fashion (from very unattractive to very attractive, very untrustworthy to very trustworthy, etc.; like we assumed before) but in an unipolar fashion relative from the mean rating. For example, this would mean that a face with an attractiveness/trusworthiness/dominance rating of -4 would be represented in the same way as a face with a rating of 4 (assuming a mean rating of 0) Moreover, suppose I believe that this representation is quadratic, not linear. For example, a face with an attractiveness rating of 2 is four times as attractive as a face with an attractiveness rating of 1. Using the social judgement data (i.e., the <tt>sj</tt> variable), create an RDM that represents this hypothesis.
</div>

In [ ]:
''' Implement the ToDo here. '''

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. '''
from niedu.tests.nipa.week_3 import test_rdm_sj2
test_rdm_sj2(sj, rdm_sj2)

RSA also lends itself very well for testing computational models, i.e., models that yield a (set of) feature(s) that were directly computed from the data. For example, in vision science, there are many computational models that yield a set of (visual) feature that are computed from the image directly (i.e., from the pixels). Again, as long as you can specify a set of features that embody your hypothesis, you can create an RDM from it! 

In the next ToDo, you're going to practice a bit to get into this "computational mindset" by applying a very simple (and theoretically meaningless) computational model to the face stimuli. In the current directory there is a subfolder `stim`, which contains all the stimuli from the first run:

In [ ]:
imgs = sorted(glob(os.path.join('stims', '*.jpg')))
print(imgs)

To load an image as a numpy array, we can use the `imageio` library. Note that the function returns a 3D numpy array, where the first two dimensions represent width and height, and the third dimension represents the three color channels (red, green, and blue).

In [ ]:
import imageio
example_stim = imageio.imread(imgs[0])
print("Shape of image data:", example_stim.shape)

<div class='alert alert-warning'>
    <b>ToDo</b> (2 points): Suppose that I have a very naive theory about color representation in the brain. Specifically, suppose that I think that a particular brain area simply represents stimuli according to their average redness, greenness, and blueness. In other words, I believe that stimuli with similar average red/green/blue values have similar brain patterns. For the forty stimuli from run 1, construct an RDM that represents this hypothesis and store it in a variable named <tt>rdm_rgb</tt> (a $40\times 40$ array). Use a Euclidean distance metric. Hint: before constructing your RDM, your feature matrix should be of shape $40 \times 3$.
</div>

In [ ]:
''' Implement your ToDo here. '''

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. '''
from niedu.tests.nipa.week_3 import test_rdm_rgb    
test_rdm_rgb(imgs, rdm_rgb)

Just by eye, it's difficult to judge whether the neural RDM contains a similar representational geometry as the feature RDMs we created earlier. Fortunately, we have statistics!

## 7. Testing RDMs
In this section, we'll discuss how to evaluate the "fit" of feature RDMs.

### 7.1. Correlation-based tests
Alright, so now we got two RDMs: the feature RDM and the neural RDM. To evaluate to what extent the two RDMs share the same representational geometry, we can simply correlate them! Before doing so, we have to do one more thing: extract the lower (or upper) triangle of the RDM. This is because RDMs are symmetric: the values above and below the diagonal are exactly the same. If we used the entire (flattened) RDM, we'd "artifically" create twice as many datapoints (i.e. the pairwise dissimilarities) than there really are, which will inflate the significance of the correlation between the RDMs because of increased sample size. So, instead of using all $N\cdot N$ pairwise differences from the RDM, we need to extract only the flattened $N\times (N-1)/2$ pairwise dissimilarity values, the "representational dissimilarity vector" (RDV) if you will. This means that we do not include the diagonal!

Fortunately, there is a function that easily extracts the lower triangle of a square distance matrix: `squareform` (from the `scipy.spatial.distance` module):

In [ ]:
from scipy.spatial.distance import squareform

# Let's extract the RDV from our neural RDM
rdv_R = squareform(rdm_R)
print("Shape rdv_R:", rdv_R.shape)

<div class='alert alert-success'>
    <b>Tip</b>: Sometimes, an RDM might not be <em>exactly</em> symmetric due to floating point inaccuracies, which will give an error my trying to extract the lower triangle using <tt>squareform</tt>. To circumvent this, you can round the RDM values to, e.g., 5 decimals using the <tt>.round(decimals</tt> array method.
</div>

As you can see, the shape of the `rdv_R` is as expected: $40 \times (40-1) / 2 = 780$. Let's do the same for the face-gender RDM we created earlier (`rdm_fg_pd`):

In [ ]:
rdv_fg = squareform(rdm_fg)
print("Shape rdv_fg:", rdv_fg.shape)

Importantly, the correlation between feature and neural RDMs is often evaluated using a rank-based correlation metric. For continuous feature RDMs, this is usually the Spearman correlation, but for categorical feature RDMs (such as our face-gender RDM), often the "Kendall Tau $\alpha$" correlation is used, as it deals properly with tied ranks. Implementations of both correlations are available from the `scipy.stats` module. Here, we'll use Kendall's Tau $\alpha$, because our face-gender RDM is categorical:

In [ ]:
from scipy.stats import kendalltau
rdm_corr, pval = kendalltau(rdv_fg, rdv_R)
print("Correlation between RDMs (p-value): %.3f (%.3f)" % (rdm_corr, pval))

Given the slighly positive correlation between our face-gender RDM and the FFA RDM, this means that patterns related to trials with the same face gender are slightly less dissimilar than patterns related to trials with a different face gender! 

### 7.2. Reweighting RDVs
One more advanced RSA technique is "reweighting". This technique allows you to use multiple feature RDVs to explain your neural RDV. Essentially, you assume that the neural RDV can be approximated as a linear weighted sum of different feature RDVs. For example, for two feature RDVs ($\mathrm{RDV}_{S_{1}}$ and $\mathrm{RDV}_{S_{2}}$):

\begin{align}
\mathrm{RDV}_{R} = \beta_{0} + \mathrm{RDV}_{S_{1}}\beta_{1} + \mathrm{RDV}_{S_{2}}\beta_{2} + \epsilon
\end{align}

You might recognize this formulation as a linear model (GLM) with the neural RDV as dependent variable and the feature RDVs as independent variables. Here, the parameters ($\beta$) represent the "reweighting" factors. This technique is very useful to disentangle the contributions of different (possibly correlated) feature spaces. Note that, often, a variant of ordinary least squares (OLS) is used to determine the parameters: non-negative least squares (NNLS), which forces the parameters to be positive (for details about why NNLS should be used, see [this article](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003915), and more information about reweighting in general, see [this article](https://www.sciencedirect.com/science/article/pii/S0028393215301998#bib5)). 

After the reweighted RDV ($\mathrm{RDV}_{S}\hat{\beta}$) is computed, it can again be evaluated using a (rank-based) correlation: $r(\mathrm{RDV}_{S}\hat{\beta}, \mathrm{RDV}_{R})$.

<div class='alert alert-warning'>
    <b>ToDo</b> (2 points): Previously, we defined two different feature RDMs: one based on face-gender (<tt>rdm_fg</tt>) and another one based on social judgements (<tt>rdm_sj</tt>). While theoretically meaningless, perform a reweighting analysis with the two feature RDVs as independent variables (in addition to an intercept!) and the neural RDV (based on <tt>rdm_R</tt>) as dependent variable. You can use the <tt>nnls</tt> implementation from <tt>scipy.optimize</tt> to perform NNLS. Note that the <tt>nnls</tt> function returns two things &mdash; the first object is the array with parameters, and it takes two arguments: the design matrix (independent variables) and the dependent variable. Compute the Spearman correlation between the reweighted RDV and the neural RDV and store the result in a variable named <tt>corr_reweighted_analysis</tt>.
</div> 

In [ ]:
from scipy.stats import spearmanr
from scipy.optimize import nnls

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. '''
from niedu.tests.nipa.week_3 import test_reweighting_todo    
test_reweighting_todo(rdm_fg, rdm_sj, rdm_R, corr_reweighted_analysis)

<div class='alert alert-info'>
    <b>ToThink</b> (2 points): Suppose that you have a very high-dimensional experimental feature space (e.g., $P=10,000$) and you decide, instead of using a single feature RDM, to use an individual RDM for each feature and analyze the neural RDM in with a reweighted analysis. Explain why, while this would guarantee amazing results, this is probably not a good idea and propose a practical solution.  
</div>

YOUR ANSWER HERE

### 7.3. Group-level analyses
So far, we only analyzed a single subject. Often, you might want to analyze multiple subjects and perform inference on the group level. Just like we did with single-subject decoding model performance scores, we can test whether results are significant by testing them against chance-level (i.e., 0 when using correlations). For example, suppose I have the following subject-specific RDV correlations:

In [ ]:
rdv_corrs = np.array([0.03, 0.01, 0.02, 0.00, 0.05, 0.07, 0.05, 0.12, 0.08, -0.07, -0.02, -0.01])

Just like we evaluated single-subject RDV correlations using a non-parameteric rank-based correlation (Spearman or Kendall's Tau), it is common to use a non-parametric test for group-level analyses: the Wilcoxon signed-rank test:

In [ ]:
from scipy.stats import wilcoxon
stat, pval = wilcoxon(rdv_corrs)
print("P-value:", pval)

<div class='alert alert-warning'>
    <b>ToDo</b> (4 points): In this ToDo, you'll implement an RSA analysis from start to end (on 10 subjects) in which you investigate whether facial expression (smiling vs. neutral) is represented in the right temporal occipital fusiform. In the end, you should have 10 RDV correlations (stored in the pre-allocated <tt>rdv_corrs</tt> variable). We have written the start of the for-loop for you already. Within the loop, do the following:

* Load in the events file as a DataFrame, extract the facial expression information (column: "expression"), convert it to a numeric format, and compute the corresponding feature RDM;
* Load in the patterns and apply the previously defined rTOF mask (the variable <tt>rTOF_roi_resamp</tt>);
* Compute the Kendall Tau correlation between the feature RDV and neural RDV and store this in the <tt>rdv_corrs</tt> array;
* After the loop, compute the associated group-level p-value using the Wilcoxon signed-rank test and store this in a variable named <tt>pval</tt>.
</div>

In [ ]:
''' Implement the ToDo here. '''

# We'll only analyze the first 10 subjects
subs = sorted([os.path.basename(d) for d in glob(os.path.join(data_dir, 'sub-*'))])[:10]

# Pre-allocate the correlation vector
rdv_corrs = np.zeros(len(subs))

# nibetaseries directory with LSA patterns (univariate noise normalized)
nibs_dir = os.path.join(data_dir, 'derivatives', 'nibetaseries_lsa_unn')

# Iterate over subjects (i = 0:9)
for i, sub in enumerate(subs):
    print("Processing %s ..." % sub)
    events_file = os.path.join(data_dir, sub, 'ses-1', 'func', sub + '_ses-1_task-face_acq-Mb4Mm27Tr700_run-1_events.tsv')
    pattern_files = sorted(glob(os.path.join(nibs_dir, sub, 'ses-1', 'func', '*run-1*MNI*STIM*betaseries.nii.gz')))
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
''' Tests the above ToDo (part 1). '''
np.testing.assert_array_almost_equal(
    np.array(
        [ 0.01048893, -0.02276479, -0.01221073,  0.01022833,  0.01670597, 
         -0.01290873,  0.04578086,  0.05264939,  0.02896321, -0.01870696]
    ),
    rdv_corrs,
    decimal=4
)
print("Well done!")

In [ ]:
''' Tests the above ToDo (part 1). '''
np.testing.assert_almost_equal(pval, 0.4446, decimal=4)
print("Well done!")

This notebook discussed the basics of representational similarity analysis. There are many more topics that we didn't discuss (such as cross-validated distance metrics, noise ceilings, and comparing different feature RDMs).

For a more in-depth technical overview of RSA, we recommend the following article:
* [Nili, H., Wingfield, C., Walther, A., Su, L., Marslen-Wilson, W., & Kriegeskorte, N. (2014). A toolbox for representational similarity analysis. PLoS computational biology, 10(4).](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003553)

(Note that the paper introduces a MATLAB toolbox for RSA; while very much in development, the authors recently rewrote the toolbox as a [Python package](https://github.com/rsagroup/pyrsa)).